# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

<b> Условия задачи: </b>

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

<b> Описание данных </b>

Данные геологоразведки трёх регионов находятся в файлах:

     datasets/geo_data_0.csv
     datasets/geo_data_1.csv
     datasets/geo_data_2.csv
Столбцы:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Описание хода работы

1. Загрузить и подготовить данные. Пояснить порядок действий.
2. Обучить и проверить модель для каждого региона:
    - Разбить данные на обучающую и валидационную выборки в соотношении 75:25.
    - Обучить модель и сделать предсказания на валидационной выборке.
    - Сохранить предсказания и правильные ответы на валидационной выборке.
    - Напечатать на экране средний запас предсказанного сырья и RMSE модели.
    - Проанализировать результаты.
3. Подготовка к расчёту прибыли:
     - Все ключевые значения для расчётов сохранить в отдельных переменных.
     - Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе. 
     - Написать выводы по этапу подготовки расчёта прибыли.
4. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
     - Выбрать скважины с максимальными значениями предсказаний. 
     - Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
     - Рассчитать прибыль для полученного объёма сырья.
5. Посчитать риски и прибыль для каждого региона:
     - Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
     - Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
     - Написать выводы: предложить регион для разработки скважин и обосновать выбор.

## Настройки рабочей тетради

In [1]:
# Импорт библиотек

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

import os

In [2]:
# Настройки библиотек

 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [3]:
# Путь к директории с данными

path = 'datasets/'

## Загрузка и подготовка данных

In [4]:
#Загрузка данных

data_1 = pd.read_csv(os.path.join(path, 'geo_data_0.csv'), index_col=0)
data_2 = pd.read_csv(os.path.join(path, 'geo_data_1.csv'), index_col=0)
data_3 = pd.read_csv(os.path.join(path, 'geo_data_2.csv'), index_col=0)

In [5]:
#Чтение данных

list_of_data = [data_1, data_2, data_3]

for value in list_of_data:
    display(value)
    value.info()

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
DLsed,0.971957,0.370953,6.075346,110.744026
QKivN,1.392429,-0.382606,1.273912,122.346843
3rnvd,1.029585,0.018787,-1.348308,64.375443


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...
QywKC,9.535637,-6.878139,1.998296,53.906522
ptvty,-10.160631,-12.558096,5.005581,137.945408
09gWa,-7.378891,-3.084104,4.998651,137.945408


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...
4GxBu,-1.777037,1.125220,6.263374,172.327046
YKFjq,-1.261523,-0.894828,2.524545,138.748846
tKPY3,-1.199934,-2.957637,5.219411,157.080080


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


Пропусков в данных нет, ошибок в типах столбцов - тоже нет. Осталось проверить данные на наличие дубликатов и масштаб признаков.

In [6]:
for i in range(len(list_of_data)):
    print('Регион №' + str(i+1))
    print('Количество дубликатов в данных равно', list_of_data[i].duplicated().sum())
    print('Максимальное значение в столбце f0 равно:', str(list_of_data[i]['f0'].max()) + '. Минимальное равно:', 
          str(list_of_data[i]['f0'].min()))
    print('Максимальное значение в столбце f1 равно:', str(list_of_data[i]['f1'].max()) + '. Минимальное равно:', 
          str(list_of_data[i]['f1'].min()))
    print('Максимальное значение в столбце f2 равно:', str(list_of_data[i]['f2'].max()) + '. Минимальное равно:', 
          str(list_of_data[i]['f2'].min()))

Регион №1
Количество дубликатов в данных равно 0
Максимальное значение в столбце f0 равно: 2.362330810854224. Минимальное равно: -1.408605306026996
Максимальное значение в столбце f1 равно: 1.343769333804496. Минимальное равно: -0.8482184970082173
Максимальное значение в столбце f2 равно: 16.003790007695365. Минимальное равно: -12.08832811806336
Регион №2
Количество дубликатов в данных равно 0
Максимальное значение в столбце f0 равно: 29.42175461390372. Минимальное равно: -31.609576019167687
Максимальное значение в столбце f1 равно: 18.73406263373076. Минимальное равно: -26.358598008345872
Максимальное значение в столбце f2 равно: 5.019720555223062. Минимальное равно: -0.0181440867134202
Регион №3
Количество дубликатов в данных равно 0
Максимальное значение в столбце f0 равно: 7.23826247979405. Минимальное равно: -8.760003624213763
Максимальное значение в столбце f1 равно: 7.844801270084258. Минимальное равно: -7.084019760867246
Максимальное значение в столбце f2 равно: 16.739402058699

<b>Вывод</b>

Загруженные данные не имеют пропусков и дубликатов, необходимо масштабирование количественных признаков, т.к. в столбцах разный разброс значений, причём у данных каждого региона они отличаются, значит стандартизировать их необходимо будет для каждых данных отдельно. 

При обучении модели, необходимо будет убрать столбец "id", т.к. это незначимый признак, который повлечёт за собой ошибки в обучении.

## Обучение и проверка модели

In [7]:
# Функция для стандартизации признаков, разделении данных и предсказаний на валидационных данных 
# с их сохранением в одном датафрейме.

def make_predictions(data):
    train, valid = train_test_split(data, test_size=0.25, random_state=12345)
    
    features_train = train.loc[:, ['f0', 'f1', 'f2']]
    features_valid = valid.loc[:, ['f0', 'f1', 'f2']]
    target_train = train['product']
    target_valid = valid['product']
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    predictions = pd.Series(predictions, index=valid.index)
    data_n = valid.join(predictions.rename('predictions'))
    
    return data_n

In [8]:
# Применение к данным функции для предссказаний сырья

data_1n = make_predictions(data_1)
data_2n = make_predictions(data_2)
data_3n = make_predictions(data_3)

display(data_1n)   #Проверка работы функции

,f0,f1,f2,product,predictions
id,,,,,
006OJ,1.289189,-0.623324,3.642039,142.822141,115.288676
009eY,1.958091,0.268711,3.702938,112.036815,105.517568
00MLz,-1.059475,0.393953,6.883800,149.442720,113.882345
00S3x,0.735088,-0.337135,0.158882,80.943922,86.298197
00Tj0,0.036445,0.086418,6.089293,147.877179,116.917069
...,...,...,...,...,...
zyk5i,-0.853286,0.708145,7.337154,135.955960,113.182719
zyksz,1.804417,-0.335788,3.493406,146.595046,112.106162
zymat,0.403372,0.873008,2.017033,96.499585,80.296981


In [9]:
# Вычисление среднего запаса предсказанного сырья и RMSE модели.

list_of_data_n = [data_1n, data_2n, data_3n]
for i in range(len(list_of_data_n)):
    rmse = mean_squared_error(list_of_data_n[i]['product'], list_of_data_n[i]['predictions']) ** 0.5
    print('Средний запас предсказанного сырья в регионе', i+1, 'равен', 
          sum(list_of_data_n[i]['predictions'])/len(list_of_data_n[i]['predictions']))
    print('RMSE модели:', rmse)

Средний запас предсказанного сырья в регионе 1 равен 92.59088494648852
RMSE модели: 37.584126870831604
Средний запас предсказанного сырья в регионе 2 равен 68.7285468954458
RMSE модели: 0.8930992867756168
Средний запас предсказанного сырья в регионе 3 равен 94.96504596800506
RMSE модели: 40.02970873393434


<b>Вывод</b>

Первый и третий регион похожи по результатам предсказаний и, несмотря на более высокое значение среднего запаса предсказанного сырья, являются довольно рискованными для разработки из-за довольно большого стандартного отклонения. Что касается второго региона, то можно сказать, что хотя средний запас предсказанного сырья меньше остальных примерно на четверть, стандартное отклонение меньше в 40 раз, значит этот регион является более перспективным, нежели первый и третий.

Если опираться только на предсказания модели, то можно сделать вывод, что второй регион более перспективный для разработки, чем первый и третий регионы.

## Подготовка к расчёту прибыли

In [10]:
# Ключевые параметры для расчетов

budget = 10000000000       # Бюджет на разработку скважин в регионе
price = 450000             # Доход с каждой единицы продукта (руб/тыс. бар)
number_wells = 500         # Количетсво скважин для разведки
number_wells_exp = 200     # Количество скважин для разработки

In [11]:
# Расчет достаточного объёма сырья для безубыточной разработки новой скважины

profit_product_well = budget/(price*number_wells_exp)
print('Достаточный объём сырья для безубыточной разработки одной новой скважины равен', profit_product_well)

Достаточный объём сырья для безубыточной разработки одной новой скважины равен 111.11111111111111


<b>Вывод</b>

Для получения прибыли необходимо, чтобы каждая новая скважина имела объём сырья для разработки больше 111 тыс. баррелей. Если посмотреть на результаты предсказаний, можно сделать вывод, что ни один регион не подходит, однако при расчёте в предыдующем пункте учитывались все скважины, а именно как "плохие" скважины (имеющие много меньший объём), так и хорошие. Нам же интересны только 200 наиболее перспективных из всех предсказаний, значит необходимо отобрать эти значения и посчитать прибыль от них.

In [12]:
# Функция для вычисления прибыли получаемой с 200 наиболее перспективных скважин в регионе

def revenue(data):
    data_k = data.sort_values('predictions', ascending=False).head(number_wells_exp)
    revenue = data_k['product'].sum()*price-budget
    return revenue

## Расчёт прибыли и рисков 

In [13]:
# Расчёт прибыли для полученного объёма сырья в каждом регионе

for i in range(len(list_of_data_n)):
    print('Прибыль для полученного сырья в регионе', i+1, 'равна', revenue(list_of_data_n[i]))

Прибыль для полученного сырья в регионе 1 равна 3320826043.1398506
Прибыль для полученного сырья в регионе 2 равна 2415086696.681511
Прибыль для полученного сырья в регионе 3 равна 2710349963.5998325


In [14]:
# Функция для нахождения распределения прибыли техникой bootstrap

def bootstrap(data):
    values = []
    state = np.random.RandomState(12345)
    for i in range(1000):
        subsample = data.sample(n=number_wells, replace=True, random_state=state)
        values.append(revenue(subsample))
    values = pd.Series(values)
    return values

In [15]:
# Нахождение средней прибыли, 95%-й доверительного интервала и риска убытков для каждого региона.

for i in range(len(list_of_data_n)):
    distr_1 = bootstrap(list_of_data_n[i])
    lower = distr_1.quantile(0.025)
    upper = distr_1.quantile(0.975)

    print('Регион №', i+1)
    print('Средняя прибыль:', sum(distr_1)/len(distr_1))
    print('95% доверительный интервал от', lower, 'до', upper)
    print('Риск убытков:', distr_1.loc[distr_1<0].count()/len(distr_1)*100, '%')

Регион № 1
Средняя прибыль: 389410689.07411075
95% доверительный интервал от -115129638.43586944 до 908468477.9631377
Риск убытков: 7.000000000000001 %
Регион № 2
Средняя прибыль: 456045105.786661
95% доверительный интервал от 33820509.39898363 до 852289453.866036
Риск убытков: 1.5 %
Регион № 3
Средняя прибыль: 404403866.56835735
95% доверительный интервал от -163350413.39560106 до 950359574.9237995
Риск убытков: 7.6 %


<b>Вывод</b>

Судя по результатам полученных при расчётах для каждого региона, второй регион является наиболее перспективным для разработки: во втором регионе наибольшее значение средней прибыли и наименьшее значение риска убытков.

Первый и третий регионы имеют немного большее значение полученной прибыли при наилучшем раскладе, чем во втором регионе: на 7% больше для первого и на 11% больше для второго, однако они также имеют и довольно большой риск убытков - примерно в 5 раз больше, что говорит о нецелесообразности их разработки при наличии второго региона